In [6]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Set the same memory growth setting for all GPUs.
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ProgbarLogger

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
import joblib # Saves and loads the model to save training time
from matplotlib import rcParams

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from sklearn.preprocessing import StandardScaler
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

import joblib
import pickle

import warnings
warnings.filterwarnings('ignore')

seed = 7
tf.random.set_seed(seed)

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
import joblib # Saves and loads the model to save training time
from matplotlib import rcParams

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from sklearn.preprocessing import StandardScaler
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

import joblib
import pickle

import warnings
warnings.filterwarnings('ignore')

seed = 7
tf.random.set_seed(seed)
import tensorflow as tf
import torch

if tf.test.is_gpu_available():
    print("GPU is available")
else:
    print("GPU is NOT available")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [8]:
#from google.colab import files
import pandas as pd

# upload the data file to Google Colab Pro
#uploaded = files.upload()

# read the data file into a Pandas DataFrame
df = pd.read_csv('Combined_data.csv', keep_default_na=False, na_values=[''])


label_encoder=LabelEncoder()
df["Surface Type"]=label_encoder.fit_transform(df["Surface Type"])
df["Material"]=label_encoder.fit_transform(df["Material"])
df["Color"]=label_encoder.fit_transform(df["Color"])
df["Manufacturer"]=label_encoder.fit_transform(df["Manufacturer"])
df["Bead Type 1"]=label_encoder.fit_transform(df["Bead Type 1"])
df["Bead Type 2"]=label_encoder.fit_transform(df["Bead Type 2"])

df.to_csv('Combined_data_encoded_ANN.csv',index=False)
df = pd.read_csv('Combined_data_encoded_ANN.csv',keep_default_na=False,na_values=[''])

df_2 = df.copy()

df_2 = df_2.astype({'NTPEP Number':'string', 
                'Sub-Deck':'string',
                'Line':'string',
                'Surface Type':'string',
                'Color':'string',
                'Material':'string',
                'Manufacturer':'string',
                #'Thickness':'string',
                'Bead Type 1':'string',
                'Bead Type 2':'string'
                })


df_2["ID"] = df_2['NTPEP Number']+df_2['Sub-Deck']+df_2['Line']+df_2['Surface Type']+df_2['Material']+df_2['Color']+df_2['Manufacturer']+df_2['Bead Type 1']+df_2['Bead Type 2']
#df_2['Thickness']


df["ID"]=df_2["ID"]

df.drop(columns=["Site","NTPEP Number","Sub-Deck","Line"],axis=1,inplace=True)

df = df [['ID', 'Surface Type', 'Material', 'Color', 'Manufacturer',
       #'Thickness', 
        'Bead Type 1', 'Bead Type 2', 'Interval', 'Traffic',
       'Rainfall', 'Snowfall', 'MRS']]

df.to_csv('Combined_data_encoded_with_ID_ANN.csv',index=False)


print(df.isnull().sum())
df.head()

ID                  0
Surface Type        0
Material            0
Color               0
Manufacturer        0
Bead Type 1         0
Bead Type 2         0
Interval            0
Traffic             0
Rainfall            0
Snowfall            0
MRS             14224
dtype: int64


,ID,Surface Type,Material,Color,Manufacturer,Bead Type 1,Bead Type 2,Interval,Traffic,Rainfall,Snowfall,MRS
0,PMM-2012-01-00161010834,0,1,0,8,3,4,0,86665.0,1.49,0.0,441.0
1,PMM-2012-01-00162010834,0,1,0,8,3,4,0,86665.0,1.49,0.0,642.0
2,PMM-2012-01-00181010834,0,1,0,8,3,4,0,86665.0,1.49,0.0,462.0
3,PMM-2012-01-00182010834,0,1,0,8,3,4,0,86665.0,1.49,0.0,481.0
4,PMM-2012-01-001629110834,1,1,0,8,3,4,0,86665.0,1.49,0.0,579.0


**Creating a function to make ANN**

In [9]:
def create_model(hidden_layers=1, nodes=16, dropout=0.0, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(nodes, input_dim=X_train.shape[1], activation='relu'))
    for i in range(hidden_layers):
        model.add(Dense(nodes, activation='relu'))
        if dropout > 0:
            model.add(Dropout(dropout))
    model.add(Dense(1, activation='linear'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

# Create the Keras wrapper for use in GridSearchCV
model = KerasRegressor(build_fn=create_model, verbose=10)

print('Model creation done')


Model creation done


**Model 1**

In [ ]:
%time 
###### Data processing for the Model 1
inputs = df.loc[df['Interval']==0]
inputs = inputs.sort_values('ID')
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall','MRS']]
outputs =  df.loc[df['Interval']==1]
outputs = outputs.sort_values('ID')
y = outputs['MRS']

##############################################################
print("ANN - Model 1 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 1 - Number of points (train):",len(X_train))##
print("ANN - Model 1 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))



Wall time: 0 ns
ANN - Model 1 - Number of points (total): 5316
ANN - Model 1 - Number of points (train): 4252
ANN - Model 1 - Number of points (test): 1064
Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV 1/10; 1/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.001, nodes=300
[CV 1/10; 1/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.001, nodes=300;, score=-4986.798 total time=  18.3s
[CV 2/10; 1/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.001, nodes=300
[CV 2/10; 1/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.001, nodes=300;, score=-9342.248 total time=  24.7s
[CV 3/10; 1/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.001, nodes=300
[CV 3/10; 1/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.001, nodes=300;, score=-3185.144 total time=  23.6s
[CV 

[CV 2/10; 4/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.01, nodes=300;, score=-11247.571 total time=   7.0s
[CV 3/10; 4/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.01, nodes=300
[CV 3/10; 4/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.01, nodes=300;, score=-5241.628 total time=   5.6s
[CV 4/10; 4/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.01, nodes=300
[CV 4/10; 4/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.01, nodes=300;, score=-5015.678 total time=  10.2s
[CV 5/10; 4/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.01, nodes=300
[CV 5/10; 4/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.01, nodes=300;, score=-7242.607 total time=   5.1s
[CV 6/10; 4/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, l

[CV 4/10; 7/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.1, nodes=300;, score=-13614.408 total time=   1.9s
[CV 5/10; 7/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.1, nodes=300
[CV 5/10; 7/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.1, nodes=300;, score=-6877.752 total time=   4.3s
[CV 6/10; 7/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.1, nodes=300
[CV 6/10; 7/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.1, nodes=300;, score=-9297.170 total time=   2.4s
[CV 7/10; 7/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.1, nodes=300
[CV 7/10; 7/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learning_rate=0.1, nodes=300;, score=-12964.396 total time=   2.6s
[CV 8/10; 7/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=1, learnin

[CV 6/10; 10/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.001, nodes=300;, score=-3929.019 total time=  18.7s
[CV 7/10; 10/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.001, nodes=300
[CV 7/10; 10/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.001, nodes=300;, score=-6087.213 total time=  18.9s
[CV 8/10; 10/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.001, nodes=300
[CV 8/10; 10/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.001, nodes=300;, score=-4575.880 total time=  25.2s
[CV 9/10; 10/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.001, nodes=300
[CV 9/10; 10/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.001, nodes=300;, score=-4224.953 total time=  24.5s
[CV 10/10; 10/72] START batch_size=128, dropout=0.0, epochs=1000, hid

[CV 8/10; 13/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.01, nodes=300;, score=-8019.911 total time=  10.0s
[CV 9/10; 13/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.01, nodes=300
[CV 9/10; 13/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.01, nodes=300;, score=-5139.933 total time=  11.1s
[CV 10/10; 13/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.01, nodes=300
[CV 10/10; 13/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.01, nodes=300;, score=-10502.229 total time=   6.2s
[CV 1/10; 14/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.01, nodes=400
[CV 1/10; 14/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.01, nodes=400;, score=-6794.136 total time=  13.5s
[CV 2/10; 14/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_l

[CV 10/10; 16/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.1, nodes=300;, score=-62103.260 total time=   6.0s
[CV 1/10; 17/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.1, nodes=400
[CV 1/10; 17/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.1, nodes=400;, score=-78076.907 total time=   9.4s
[CV 2/10; 17/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.1, nodes=400
[CV 2/10; 17/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.1, nodes=400;, score=-88834.534 total time=   7.7s
[CV 3/10; 17/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.1, nodes=400
[CV 3/10; 17/72] END batch_size=128, dropout=0.0, epochs=1000, hidden_layers=2, learning_rate=0.1, nodes=400;, score=-67405.473 total time=   7.7s
[CV 4/10; 17/72] START batch_size=128, dropout=0.0, epochs=1000, hidden_layers

In [ ]:
model_1_tuned = grid_result.best_estimator_.model
# Saving the model
model_1_tuned.save("ANN_model_1.h5")

# Accuracy with the train data
y_pred_train_f = model_1_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_1_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_1 = model_1_tuned.predict(X_scaled) ##

# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS1"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_1 = pd.concat([df1, df2], axis=1)##
df_test_model_1.to_csv('ANN_Model_1_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS1"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_1 = pd.concat([df3, df4], axis=1) ##
df_train_model_1.to_csv('ANN_Model_1_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'MRS']
explainer = shap.DeepExplainer(model_1_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

#shap.initjs()
#shap.force_plot(explainer.expected_value[0],shap_values[0][0],features=Predictors)

In [ ]:
feedrow_model_1 = [[1,4,0,12,0,4,3941418,0.67,0.6,751]]
feedrow_model_1 = scaler.transform(feedrow_model_1)
predictedoutput_model_1 = model_1_tuned.predict([feedrow_model_1])
print("Predicted RS at Month 1:",predictedoutput_model_1)

**Model 2**

In [ ]:
%time 
###############################################################
inputs = df.loc[df['Interval']==1]
inputs = inputs.sort_values('ID')
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1',
       'Bead Type 2', 'Traffic','Rainfall','Snowfall']]
X['PRS'] = y_pred_1
X = X.reset_index()
X = X.drop(['index'],axis=1)

outputs =  df.loc[df['Interval']==2]
outputs = outputs.sort_values('ID')
outputs = outputs.reset_index()
outputs = outputs.drop(['ID'],axis=1)

X['MRS'] = outputs['MRS']
null_indexes = X[X['MRS'].isnull()].index.tolist()
X = X.dropna()
y = X['MRS']
X = X.drop(['MRS'],axis=1)
##############################################################
print("ANN - Model 2 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 2 - Number of points (train):",len(X_train))##
print("ANN - Model 2 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_2_tuned = grid_result.best_estimator_.model

# Saving the model
model_2_tuned.save("ANN_model_2.h5")

# Accuracy with the train data
y_pred_train_f = model_2_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_2_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_2 = model_2_tuned.predict(X_scaled) ##
##########################################################
X['PRS'] = y_pred_2 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_2= X['PRS'] ####
y_pred_2 = y_pred_2.sort_index() ###
##########################################################

# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS2"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_1 = pd.concat([df1, df2], axis=1)##
df_test_model_1.to_csv('ANN_Model_2_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS2"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_1 = pd.concat([df3, df4], axis=1) ##
df_train_model_1.to_csv('ANN_Model_2_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_2_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

#shap.initjs()
#shap.force_plot(explainer.expected_value[0],shap_values[0][0],features=Predictors)

In [ ]:
###
feedrow_model_2 = [[1,4,0,12,0,4,4162977,0.81,0.6,predictedoutput_model_1]]
feedrow_model_2 = scaler.transform(feedrow_model_2)
predictedoutput_model_2 = model_2_tuned.predict(feedrow_model_2) ##
print("Predicted RS at Month 2:",predictedoutput_model_2)##

**Model 3**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==2]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1',
       'Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_2.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==3] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 3 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 3 - Number of points (train):",len(X_train))##
print("ANN - Model 3 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_3_tuned = grid_result.best_estimator_.model

# Saving the model
model_3_tuned.save("ANN_model_3.h5")

# Accuracy with the train data
y_pred_train_f = model_3_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_3_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_3 = model_3_tuned.predict(X_scaled) ##
##########################################################
X['PRS'] = y_pred_3 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_3 = X['PRS'] ####
y_pred_3 = y_pred_3.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS3"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_3 = pd.concat([df1, df2], axis=1)##
df_test_model_3.to_csv('ANN_Model_3_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS3"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_3 = pd.concat([df3, df4], axis=1) ##
df_train_model_3.to_csv('ANN_Model_3_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_3_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_3 = [[1,4,0,12,0,4,4477824,0.92,0.6,predictedoutput_model_2]]
feedrow_model_3 = scaler.transform(feedrow_model_3)
predictedoutput_model_3 = model_3_tuned.predict(feedrow_model_3) ##
print("Predicted RS at Month 3:",predictedoutput_model_3)##

**Model 4**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==3]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_3.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==11] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 4 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 4 - Number of points (train):",len(X_train))##
print("ANN - Model 4 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_4_tuned = grid_result.best_estimator_.model

# Saving the model
model_4_tuned.save("ANN_model_4.h5")

# Accuracy with the train data
y_pred_train_f = model_4_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_4_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_4 = model_4_tuned.predict(X_scaled) ##

##########################################################
X['PRS'] = y_pred_4 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_4= X['PRS'] ####
y_pred_4 = y_pred_4.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS11"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_4 = pd.concat([df1, df2], axis=1)##
df_test_model_4.to_csv('ANN_Model_4_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS11"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_4 = pd.concat([df3, df4], axis=1) ##
df_train_model_4.to_csv('ANN_Model_4_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_4_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_4 = [[1,4,0,12,0,4,4897620,0.96,0.6,predictedoutput_model_3]]
feedrow_model_4 = scaler.transform(feedrow_model_4)
predictedoutput_model_4= model_4_tuned.predict([feedrow_model_4])
print("Predicted RS at Month 11:",predictedoutput_model_4)

**Model 5**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==11]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_4.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==12] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 5 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 5 - Number of points (train):",len(X_train))##
print("ANN - Model 5 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_5_tuned = grid_result.best_estimator_.model

# Saving the model
model_5_tuned.save("ANN_model_5.h5")

# Accuracy with the train data
y_pred_train_f = model_5_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_5_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_5 = model_5_tuned.predict(X_scaled) ##

##########################################################
X['PRS'] = y_pred_5 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_5= X['PRS'] ####
y_pred_5 = y_pred_5.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS12"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_5 = pd.concat([df1, df2], axis=1)##
df_test_model_5.to_csv('ANN_Model_5_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS12"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_5 = pd.concat([df3, df4], axis=1) ##
df_train_model_5.to_csv('ANN_Model_5_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_5_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_5 = [[1,4,0,12,0,4,7684599,1.51,2.2,predictedoutput_model_4]]
feedrow_model_5 = scaler.transform(feedrow_model_5)
predictedoutput_model_5 = model_5_tuned.predict([feedrow_model_5])
print("Predicted RS at Month 12:",predictedoutput_model_5[0])

**Model 6**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==12]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_5.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==15] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 6 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 6 - Number of points (train):",len(X_train))##
print("ANN - Model 6 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_6_tuned = grid_result.best_estimator_.model

# Saving the model
model_6_tuned.save("ANN_model_6.h5")

# Accuracy with the train data
y_pred_train_f = model_6_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_6_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)
#Making final prediction
y_pred_6 = model_6_tuned.predict(X_scaled) ##
##########################################################
X['PRS'] = y_pred_6 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_6= X['PRS'] ####
y_pred_6 = y_pred_6.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS15"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_6 = pd.concat([df1, df2], axis=1)##
df_test_model_6.to_csv('ANN_Model_6_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS15"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_6 = pd.concat([df3, df4], axis=1) ##
df_train_model_6.to_csv('ANN_Model_6_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_6_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_6 = [[1,4,0,12,0,4,8174361,1.79,2.2,predictedoutput_model_5]]
feedrow_model_6 = scaler.transform(feedrow_model_6)
predictedoutput_model_6= model_6_tuned.predict([feedrow_model_6])
print("Predicted RS at Month 15:",predictedoutput_model_6)

**Model 7**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==15]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_6.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==21] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 7 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 7 - Number of points (train):",len(X_train))##
print("ANN - Model 7 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_7_tuned = grid_result.best_estimator_.model

# Saving the model
model_7_tuned.save("ANN_model_7.h5")

# Accuracy with the train data
y_pred_train_f = model_7_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_7_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_7 = model_7_tuned.predict(X_scaled) ##

##########################################################
X['PRS'] = y_pred_7 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_7= X['PRS'] ####
y_pred_7 = y_pred_7.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS21"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_7 = pd.concat([df1, df2], axis=1)##
df_test_model_7.to_csv('ANN_Model_7_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS21"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_7 = pd.concat([df3, df4], axis=1) ##
df_train_model_7.to_csv('ANN_Model_7_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_7_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_7 = [[1,4,0,12,0,4,9538698,2.2,2.2,predictedoutput_model_6]]
feedrow_model_7 = scaler.transform(feedrow_model_7)
predictedoutput_model_7= model_7_tuned.predict([feedrow_model_7])
print("Predicted RS at Month 21:",predictedoutput_model_7)

**Model 8**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==21]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_7.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==24] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 8 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 8 - Number of points (train):",len(X_train))##
print("ANN - Model 8 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_8_tuned = grid_result.best_estimator_.model

# Saving the model
model_8_tuned.save("ANN_model_8.h5")

# Accuracy with the train data
y_pred_train_f = model_8_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_8_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_8 = model_8_tuned.predict(X_scaled) ##

##########################################################
X['PRS'] = y_pred_8 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_8= X['PRS'] ####
y_pred_8 = y_pred_8.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS24"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_8 = pd.concat([df1, df2], axis=1)##
df_test_model_8.to_csv('ANN_Model_8_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS24"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_8 = pd.concat([df3, df4], axis=1) ##
df_train_model_8.to_csv('ANN_Model_8_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_8_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_8 = [[1,4,0,12,0,4,11824254,2.74,3.3,predictedoutput_model_7]]
feedrow_model_8 = scaler.transform(feedrow_model_8)
predictedoutput_model_8= model_8_tuned.predict([feedrow_model_8])
print("Predicted RS at Month 24:",predictedoutput_model_8)

**Model 9**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==24]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_8.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==27] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 9 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 9 - Number of points (train):",len(X_train))##
print("ANN - Model 9 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_9_tuned = grid_result.best_estimator_.model

# Saving the model
model_9_tuned.save("ANN_model_9.h5")

# Accuracy with the train data
y_pred_train_f = model_9_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_9_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)
#Making final prediction
y_pred_9 = model_9_tuned.predict(X_scaled) ##

##########################################################
X['PRS'] = y_pred_9 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_9= X['PRS'] ####
y_pred_9 = y_pred_8.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS27"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_9 = pd.concat([df1, df2], axis=1)##
df_test_model_9.to_csv('ANN_Model_9_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS27"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_9 = pd.concat([df3, df4], axis=1) ##
df_train_model_9.to_csv('ANN_Model_9_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_9_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_9 = [[1,4,0,12,0,4,12407304,2.86,3.3,predictedoutput_model_8]]
feedrow_model_9 = scaler.transform(feedrow_model_9)
predictedoutput_model_9= model_9_tuned.predict([feedrow_model_9])
print("Predicted RS at Month 27:",predictedoutput_model_9)

**Model 10**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==27]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_9.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==33] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 10 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 10 - Number of points (train):",len(X_train))##
print("ANN - Model 10 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_10_tuned = grid_result.best_estimator_.model

# Saving the model
model_10_tuned.save("ANN_model_10.h5")

# Accuracy with the train data
y_pred_train_f = model_10_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_10_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)
#Making final prediction
y_pred_10 = model_10_tuned.predict(X_scaled) ##

##########################################################
X['PRS'] = y_pred_10 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_10= X['PRS'] ####
y_pred_10 = y_pred_10.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS33"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_10 = pd.concat([df1, df2], axis=1)##
df_test_model_10.to_csv('ANN_Model_10_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS33"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_10 = pd.concat([df3, df4], axis=1) ##
df_train_model_10.to_csv('ANN_Model_10_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_10_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_10 = [[1,4,0,12,0,4,13864929,3.23,3.3,predictedoutput_model_9]]
feedrow_model_10 = scaler.transform(feedrow_model_10)
predictedoutput_model_10 = model_10_tuned.predict([feedrow_model_10])
print("Predicted RS at Month 33:",predictedoutput_model_10)

**Model 11**

In [ ]:
%time 
####################################################################
inputs = df.loc[df['Interval']==33]  
inputs = inputs.sort_values('ID') 
X = inputs[['Surface Type','Material','Color','Manufacturer',#'Thickness',
            'Bead Type 1','Bead Type 2', 'Traffic','Rainfall','Snowfall']] 

X['PRS'] = y_pred_10.values 
X = X.reset_index() 
X = X.drop(['index'],axis=1) 

outputs =  df.loc[df['Interval']==36] # Getting the data from interval 3 (make it 2)
outputs = outputs.sort_values('ID')#Sorting wrt UI
outputs = outputs.reset_index() # Reseting index
outputs = outputs.drop(['index'],axis=1) # dropping the index column
   
X['MRS'] = outputs['MRS'] # creating a column in X with the measured values from interval 3. This
null_indexes1 = X[X['MRS'].isnull()].index.tolist()
null_indexes2 = X[X['PRS'].isnull()].index.tolist()
# column has NaN
#drop nas
X = X.dropna() # Dropping NaN from RS column
y = X['MRS'] # putting RS from X to y
X = X.drop(['MRS'],axis=1) #Dropping RS column from X
null_indexes = null_indexes1 + null_indexes2
######################################################################
print("ANN - Model 11 - Number of points (total):",len(X))## 

# Splitting data into test and train
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
print("ANN - Model 11 - Number of points (train):",len(X_train))##
print("ANN - Model 11 - Number of points (test):",len(X_test))##

# Scale the predictors using standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'nodes': [300, 400, 500],
    'dropout': [0.0, 0.05, 0.1,0.2],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size':[128],                  #[32, 64, 128],
    'epochs': [1000]
}

# Perform grid search with 10-fold cross validation
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='neg_mean_squared_error', 
                    verbose=10)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
grid_result = grid.fit(X_train_scaled, 
                       y_train, 
                       validation_split = 0.2,
                       callbacks = [early_stopping, ProgbarLogger()],
                       verbose=0)

# Print the best hyperparameters and the corresponding R2 score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

model_11_tuned = grid_result.best_estimator_.model

# Saving the model
model_11_tuned.save("ANN_model_11.h5")

# Accuracy with the train data
y_pred_train_f = model_11_tuned.predict(X_train_scaled)
print("\nTrain set evaluation:")
print("R2 score:", r2_score(y_train, y_pred_train_f))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train_f)))
print("MAPE:", mean_absolute_percentage_error(y_train, y_pred_train_f))
n = len(X_train)
p = X_train.shape[1]
adj_r2_train = 1 - (1 - r2_score(y_train, y_pred_train_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_train)

# Accuracy with the test data
y_pred_test_f = model_11_tuned.predict(X_test_scaled)
print("\nTest set evaluation:")
print("R2 score:", r2_score(y_test, y_pred_test_f))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test_f)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred_test_f))
n = len(X_test)
p = X_test.shape[1]
adj_r2_test = 1 - (1 - r2_score(y_test, y_pred_test_f)) * (n - 1) / (n - p - 1)
print("Adjusted R2 score:", adj_r2_test)

#Making final prediction
y_pred_11 = model_11_tuned.predict(X_scaled) ##

##########################################################
X['PRS'] = y_pred_11 ####
for indexes in null_indexes:
    X.loc[indexes,'PRS'] = np.nan
y_pred_11= X['PRS'] ####
y_pred_11 = y_pred_11.sort_index() ###
##########################################################
# Saving results
df1 = pd.DataFrame(y_test)
df1 = df1.reset_index(drop=True)
df2 = pd.DataFrame(y_pred_test_f,columns=["PRS36"]) ##
df2 = df2.reset_index(drop=True)
df_test_model_11 = pd.concat([df1, df2], axis=1)##
df_test_model_11.to_csv('ANN_Model_11_test.csv',index=False) ##

df3 = pd.DataFrame(y_train)
df3 = df3.reset_index(drop=True)
df4 = pd.DataFrame(y_pred_train_f,columns=["PRS36"]) ##
df4 = df4.reset_index(drop=True)
df_train_model_11 = pd.concat([df3, df4], axis=1) ##
df_train_model_11.to_csv('ANN_Model_11_train.csv',index=False) ##

In [ ]:
import shap
Predictors=['Surface Type', 'Material', 'Color', 'Manufacturer', 'Bead Type 1', 
            'Bead Type 2', 'Traffic', 'Rainfall', 'Snowfall', 'PRS']
explainer = shap.DeepExplainer(model_11_tuned, X_train_scaled)
shap_values = explainer.shap_values(X_test_scaled)
shap.summary_plot(shap_values, X_test_scaled, feature_names=Predictors)

In [ ]:
###
feedrow_model_11 = [[1,4,0,12,0,4,16068858,3.95,4.2,predictedoutput_model_10]]
feedrow_model_11 = scaler.transform(feedrow_model_11)
predictedoutput_model_11= model_11_tuned.predict([feedrow_model_11])
print("Predicted RS at Month 36:",predictedoutput_model_11)

**Model 1 Checking**

In [ ]:
model_1_check = keras.Sequential([
    keras.layers.Dense(units=300, activation='relu', input_dim=X_train_scaled.shape[1]),
    keras.layers.Dropout(0.0),
    keras.layers.Dense(units=1, activation='linear')
])
model_1_check.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model_1_check.fit(X_train_scaled, y_train, validation_split=0.2, epochs=50, batch_size=16, callbacks=[early_stopping])

# Plot the learning curves
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()